In [ ]:
import requests
from bs4 import BeautifulSoup
import pickle
import pandas as pd

In [ ]:
# リンク一覧リスト
atc_link = pickle.load(open('./attackdex_link_list.pkl', 'rb'))

In [ ]:
# タイプ一覧データフレーム
df_type = pd.read_csv('./type.csv')

In [ ]:
# カテゴリー一覧データフレーム
df_atc_cat = pd.read_csv('./attack_category.csv')

In [ ]:
# 国別の技名一覧データフレーム
df_atc_name = pd.read_csv('./attackdex_name.csv')

In [ ]:
# 初期化処理：アタックデックス用データフレームの生成
df_attackdex = pd.DataFrame(columns=['English_name', 'Japanese_name', 'Type', 'Category', 'PP', 'Power', 'Accuracy'])

In [ ]:
for atc_item in atc_link:
    # ページ情報の取得
    atc_url = f'https://www.serebii.net{atc_item}'
    res = requests.get(atc_url)
    soup = BeautifulSoup(res.text, 'html.parser')

    # 英語名の取得
    english_name = soup.select('.dextable td.cen')[0].contents[0].get_text(strip=True)

    # 日本語名の取得（無い場合は空文字''が代入される）
    japanese_name = soup.select('.dextable td.cen')[0].contents[2].get_text(strip=True)

    # タイプの取得
    type_link = soup.select('.dextable td.cen')[1].find('a').attrs['href'].split('/')[-1]
    type_val = df_type.query(f'link_text == "{type_link}"')['japanese'].values[0]

    # カテゴリーの取得
    category_link = soup.select('.dextable td.cen')[2].find('a').attrs['href'].split('/')[-1]
    category_val = df_atc_cat.query(f'link_text == "{category_link}"')['japanese'].values[0]

    # PPの取得
    pp_val = soup.select('.dextable td.cen')[3].get_text(strip=True)

    # 威力の取得
    power_val = soup.select('.dextable td.cen')[4].get_text(strip=True)

    # 命中率の取得
    accuracy_val = soup.select('.dextable td.cen')[5].get_text(strip=True)

    # データフレームに追加
    params = {
        'English_name': english_name,
        'Japanese_name': japanese_name,
        'Type': type_val,
        'Category': category_val,
        'PP': pp_val,
        'Power': power_val,
        'Accuracy': accuracy_val
    }
    df_attackdex = df_attackdex.append(params, ignore_index=True)

In [ ]:
# 作成したアタックデックス用データフレームのコピーを作成（念のため）
dfc_attackdex = df_attackdex.copy()

In [ ]:
# 日本語の無い部分を補完
# エラーが出た場合は手動で入力
for i in dfc_attackdex[dfc_attackdex['Japanese_name'] == ''].index.tolist():
    dfc_attackdex.loc[i, ['Japanese_name']] = df_atc_name.loc[df_atc_name['英語'] == dfc_attackdex.loc[i, ['English_name']].values[0], ['日本語']].values[0][0]

In [ ]:
# 日本語名を手動で入力する方法
# 技名参考サイト：https://wiki.xn--rckteqa2e.com/wiki/%E3%82%8F%E3%81%96%E3%81%AE%E5%A4%96%E5%9B%BD%E8%AA%9E%E5%90%8D%E4%B8%80%E8%A6%A7

# dfc_attackdex.loc[インデックス, ['Japanese_name']] = '技名'
# 例）dfc_attackdex.loc[415, ['Japanese_name']] = 'てんこがすめつぼうのひかり'

In [ ]:
# CSVに出力
dfc_attackdex.to_csv('./attackdex.csv', index=False)